In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
import pandas as pd
import os
import openai
from langchain.vectorstores import Pinecone
import pinecone

In [2]:
model_name = "shibing624/text2vec-base-chinese"

embed = SentenceTransformerEmbeddings(model_name=model_name)

/data/code/ChatDrug/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
index_name = "langchain-chatdrug"

In [5]:
texts_field = "text"

index = pinecone.Index(index_name)
vectorstore = Pinecone(
    index=index, embedding_function=embed.embed_query, text_key=texts_field
)

In [7]:
pinecone.init(
    api_key=os.getenv("PINECONE_TOKEN"), environment=os.getenv("PINECONE_LOC")
)

In [9]:
query = "结肠癌吃什么药"

vectorstore.similarity_search(query, k=3)

MaxRetryError: HTTPSConnectionPool(host='langchain-chatdrug-unknown.svc.us-west1-gcp.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [37]:
llm = ChatOpenAI(temperature=0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever()
)

In [38]:
qa.run(query)

'根据提供的上下文，可以使用多种药物治疗结肠癌，包括但不限于本品。然而，具体的治疗方案应该由医生根据患者的情况进行制定。如果您怀疑自己患有结肠癌，请尽快咨询专业医生的建议。'

In [39]:
from langchain import PromptTemplate

In [44]:
print(qa.combine_documents_chain.llm_chain.prompt)

input_variables=['context', 'question'] output_parser=None partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]


In [50]:
temp = """
从下面内容中回答用户问题，如果出现类似本品等表述，需要找出其指代的药品。如果你不知道答案，就说你不知道，不要瞎编乱造。
{summaries}
{question}
"""

In [52]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=temp, input_variables=["summaries", "question"]
        )
    },
)

ValidationError: 1 validation error for StuffDocumentsChain
__root__
  document_variable_name context was not found in llm_chain input_variables: ['summaries', 'question'] (type=value_error)